# Battle of the Neighbourhoods - Holiday Destinations

![Alt text](Coursera_Capstone/Capture.JPG)

## Project Description:

While this was supposed to be a project to compare two neighbourhoods, I decided that in my life it would be more useful to try to analyse and understand which holiday would suit us the best.

### Description of the problem:

Last year when we went on holiday to Devon for the 4th year running, we went to the beach and my wife, who's never that enthusiastic about beach trips, came along with us. We built our sand castle while Mrs D sat down and started to read her book. 
As the wind was blowing and we'd bought a kite, we boys went off and flew it and were off across the beach having fun. When we got back we couldn't find Mrs D but there was the remains of our sand castle :( 
Mrs D turned up soaking wet and explained that the tide had come in and caught her completely unawares and that she and her soggy book were going back to the holiday cottage right now.
So all future holidays, she says, will be city breaks.
### Analysis Saves the day (hopefully)

This project will search potential holiday destinations and score them for desirability based on the requirements of all of the family. I will be searching for the top 100 attractions from centroid location in a city and assigning each a score. 
This will give a better understanding of what is on offer within walking distance of each centre and enable me to make a case ~~that there really isn't anything there for children and that we should go to the sea-side instead~~ for a fairer choice of holiday destination

## Methodology

First we'll identify 3 holiday destination coordinates using Geopy and our original tried and tested holiday destination, Devon, for comparison purposes:

In [36]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import warnings
warnings.filterwarnings('ignore')
address = 'Devon'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of {} is {}, {}.'.format(address, latitude_x, longitude_y))

The geograpical coordinate of Devon is 50.75, -3.75.


and plot them on a map to visually detail the scope:

In [2]:
import folium

In [37]:
m = folium.Map(
    location=[52.3745403, 4.89797550561798],
    zoom_start=5,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[55.6867243, 12.5700724],
    popup='Copenhagen',
    icon=folium.Icon(icon='flag')
).add_to(m)

folium.Marker(
    location=[48.85341, 2.3488],
    popup='Paris',
    icon=folium.Icon(color='green', icon='flag')
).add_to(m)

folium.Marker(
    location=[52.6843696, -1.8275286],
    popup='Home',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

folium.Marker(
    location=[52.3745403, 4.89797550561798],
    popup='Amsterdam',
    icon=folium.Icon(color='orange', icon='flag')
).add_to(m)

folium.Marker(
    location=[50.75, -3.75],
    popup='Devon',
    icon=folium.Icon(color='red', icon='heart')
).add_to(m)

m

## Next steps

I'm next going to use these coordinates in a Foursquare query to find venues, hopefully restricted to holiday attractions, though I haven't figured that out yet, and display these with appropriate markers onto a map.

Then determine scores for each of these categories for the 4 members of the family (M, A, Z and E) and put these into a DataFrame

Combining the data from FourSquare and the scores from the family, calculate the desirability of the 4 holiday destinations: Devon (control), Paris, Copenhagen and Amsterdam.

Then use clusting analysis to ensure that each member of the family will enjoy the holiday and prepare a presentation back to Mrs D to get her buy in on the decision made.

### Rough code which I used to scope out feasibility of project:

In [ ]:
# find distinct venues in Paris as sample set
# perhaps filter for only hoilday relevant venues
# determine scoing from 4 parties 
# describe calcaultions
# plot locations based desirabilities back to map

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [32]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_x, 
   longitude_y, 
    radius, 
   LIMIT)

In [33]:
results = requests.get(url).json()

In [34]:
results

{'meta': {'code': 200, 'requestId': '5cfad0d3351e3d128475bb62'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Paris',
  'headerFullLocation': 'Paris',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 48.91961016300006,
    'lng': 2.4470730930644926},
   'sw': {'lat': 48.79361003699994, 'lng': 2.2559253069355076}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf41231e5eba59334341f90',
       'name': "Place de l'Hôtel de Ville – Esplanade de la Libération",
       'location': {'address': "Place de l'Hôtel de Ville",
        'lat': 48.85713761176484,
        'lng': 2.3516931321499746,
        'distance': 60,
        'postalCode':

In [36]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bf41231e5eba59334341f90-0,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",4bf41231e5eba59334341f90,Place de l'Hôtel de Ville,FR,Paris,France,NaN,60,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,NaN,75004,Île-de-France,Place de l'Hôtel de Ville – Esplanade de la Li...,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b533b45f964a520509327e3-1,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",4b533b45f964a520509327e3,39 et 52 rue Sainte-Croix de la Bretonnerie,FR,Paris,France,NaN,308,"[39 et 52 rue Sainte-Croix de la Bretonnerie, ...","[{'label': 'display', 'lat': 48.85876287681194...",48.858763,2.354161,NaN,75004,Île-de-France,Fleux',0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adcda09f964a520e83321e3-2,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",4adcda09f964a520e83321e3,6 place du parvis Notre-Dame,FR,Paris,France,Place Jean-Paul II,413,[6 place du parvis Notre-Dame (Place Jean-Paul...,"[{'label': 'display', 'lat': 48.85312443201169...",48.853124,2.349561,NaN,75004,Île-de-France,Cathédrale Notre-Dame de Paris,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5113ebbce4b0d3c1022aef14-3,"[{'id': '52f2ab2ebcbc57f1066b8b1b', 'name': 'S...",5113ebbce4b0d3c1022aef14,Place Georges Pompidou,FR,Paris,France,NaN,460,"[Place Georges Pompidou, 75004 Paris, France]","[{'label': 'display', 'lat': 48.86070574305312...",48.860706,2.352362,NaN,75004,Île-de-France,Boutique du Centre Pompidou,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bb5b356941ad13a31761ee3-4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",4bb5b356941ad13a31761ee3,6 rue Sainte-Croix de la Bretonnerie,FR,Paris,France,NaN,443,"[6 rue Sainte-Croix de la Bretonnerie, 75004 P...","[{'label': 'display', 'lat': 48.85771855127609...",48.857719,2.357322,NaN,75004,Île-de-France,Les Mots à la Bouche,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-512534ece4b03a94f9a50313-5,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",512534ece4b03a94f9a50313,51 rue du Temple,FR,Paris,France,Rue des Blancs-Manteaux,428,"[51 rue du Temple (Rue des Blancs-Manteaux), 7...","[{'label': 'display', 'lat': 48.859958, 'lng':...",48.859958,2.354374,NaN,75004,Île-de-France,Comptoir Gourmet,0,[],49532269
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adcda21f964a520f23921e3-6,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",4adcda21f964a520f23921e3,37 rue de la Bûcherie,FR,Paris,France,NaN,553,"[37 rue de la Bûcherie, 75005 Paris, France]","[{'label': 'display', 'lat': 48.85256839809226...",48.852568,2.347096,NaN,75005,Île-de-France,Shakespeare & Company,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4adcda0af964a520623421e3-7,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",4adcda0af964a520623421e3,Place Georges Pompidou,FR,Paris,France,Rue Beaubourg,458,"[Place Georges Pompidou (Rue Beaubourg), 75004...","[{'label': 'display', 'lat': 48.86072970727395...",48.860730,2.351660,NaN,75004,Île-de-France,Centre Pompidou – Musée National d'Art Moderne,0,[],NaN
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b27effef964a5209a8c24e3-8,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",4b27effef964a5209a8c24e3,17 rue des Lavandières Sainte-Opportune,FR,Paris,France,Rue de Rivoli,468,[17 rue des Lavandières Sainte-Opportune (Rue ...,"[{'label': 'display', 'lat': 48.85889313897845...",48.858893,2.346129,NaN,75001,Î

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Place de l'Hôtel de Ville – Esplanade de la Li...,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",48.857138,2.351693
1,Fleux',"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",48.858763,2.354161
2,Cathédrale Notre-Dame de Paris,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",48.853124,2.349561
3,Boutique du Centre Pompidou,"[{'id': '52f2ab2ebcbc57f1066b8b1b', 'name': 'S...",48.860706,2.352362
4,Les Mots à la Bouche,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",48.857719,2.357322
5,Comptoir Gourmet,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",48.859958,2.354374
6,Shakespeare & Company,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",48.852568,2.347096
7,Centre Pompidou – Musée National d'Art Moderne,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",48.860730,2.351660
8,Au Vieux Comptoir,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",48.858893,2.346129
9,François Pralus,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",48.861072,2.353806


In [39]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,48.857138,2.351693
1,Fleux',Furniture / Home Store,48.858763,2.354161
2,Cathédrale Notre-Dame de Paris,Church,48.853124,2.349561
3,Boutique du Centre Pompidou,Souvenir Shop,48.860706,2.352362
4,Les Mots à la Bouche,Bookstore,48.857719,2.357322
5,Comptoir Gourmet,Restaurant,48.859958,2.354374
6,Shakespeare & Company,Bookstore,48.852568,2.347096
7,Centre Pompidou – Musée National d'Art Moderne,Art Museum,48.860730,2.351660
8,Au Vieux Comptoir,French Restaurant,48.858893,2.346129
9,François Pralus,Gourmet Shop,48.861072,2.353806


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
        CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
paris_venues = getNearbyVenues(names=df_chicago['Neighborhood'],
                                   latitudes=df_chicago['Latitude'],
                                   longitudes=df_chicago['Longitude']
                                  )

In [ ]:
paris_venues.groupby('Neighborhood').count().head()

In [ ]:
# One hot encoding

paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]
paris_onehot.head()

In [ ]:
paris_onehot.shape

In [ ]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped.head()

In [ ]:
paris_grouped.shape

In [ ]:

num_top_venues = 5
for hood in paris_grouped['Neighborhood'][:3]:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')